# Planning Pattern - ReAct Technique

<img src="../img/planning_pattern.png" alt="Alt text" width="500"/>

---

So, we've seen agents capable of reflecting and using tools to access the outside world. But ... **what about planning**,
i.e. deciding what sequence of steps to follow to accomplish a large task?

That is exactly what the Planning Pattern provides; ways for the LLM to break a task into **smaller, more easily accomplished subgoals** without losing track of the end goal.

The most paradigmatic example of the planning pattern is the [**ReAct**](https://react-lm.github.io/) technique, displayed in the diagram above.

In this notebook, you'll learn how this technique actually works. This is the **third lesson** of the "Agentic Patterns from Scratch" series. Take a look
at the previous lessons if you haven't!

* [First Lesson: The Reflection Pattern](https://github.com/neural-maze/agentic_patterns/blob/main/notebooks/reflection_pattern.ipynb)
* [Second Lesson: The Tool Pattern](https://github.com/neural-maze/agentic_patterns/blob/main/notebooks/tool_pattern.ipynb)

## Relevant imports and Groq Client

We start by importing all the libraries we'll be using in this tutorial as well as the Groq client.

In [ ]:
import os
import re
import math
import json
from dotenv import load_dotenv

from groq import Groq

from agentic_patterns.tool_pattern.tool import tool
from agentic_patterns.utils.extraction import extract_tag_content


# Remember to load the environment variables. You should have the Groq API Key in there :)
load_dotenv()

MODEL = "llama-3.3-70b-versatile"
GROQ_CLIENT = Groq()

> If you are not familiar with the `tool` decorator, changes are you are missed the previous tutorial about the Tool Pattern. Check the video [here](https://www.youtube.com/watch?v=ApoDzZP8_ck&t=671s&ab_channel=TheNeuralMaze).

## A System Prompt for the ReAct Loop

As we did with the Tool Pattern, we also need a System Prompt for the ReAct technique. This System Prompt is very similar, the difference is that it describes the ReAct loop, so that the LLM is aware of
the three operations it's allowed to use:

1. Thought: The LLM will think about which action to take
2. Action: The LLM will use a Tool to "act on the environment"
3. Observation: The LLM will observe the tool output and reflect on the next thing to do.

Another key difference from the Tool Pattern System Prompt is that we are going to enclose all the messages with tags, like these: <thought></thought>, <observation></observation>. We could implement the ReAct logic without these tags, but I found it eeasier for the LLM to understand the instructions this way.

Ok! So without further ado, there's the prompt!

In [ ]:
# Define the System Prompt as a constant
REACT_SYSTEM_PROMPT = """
You are a function calling AI model. You operate by running a loop with the following steps: Thought, Action, Observation.
You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don' make assumptions about what values to plug
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.

For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>, "id": <monotonically-increasing-id>}
</tool_call>

Here are the available tools / actions:

<tools> 
%s
</tools>

Example session:

<question>What's the current temperature in Madrid?</question>
<thought>I need to get the current weather in Madrid</thought>
<tool_call>{"name": "get_current_weather","arguments": {"location": "Madrid", "unit": "celsius"}, "id": 0}</tool_call>

You will be called again with this:

<observation>{0: {"temperature": 25, "unit": "celsius"}}</observation>

You then output:

<response>The current temperature in Madrid is 25 degrees Celsius</response>

Additional constraints:

- If the user asks you something unrelated to any of the tools above, answer freely enclosing your answer with <response></response> tags.
"""

## Example step by step

### Defining the Tools

Let's build an example that involves the use of three tools, like the following ones.

In [ ]:
@tool
def sum_two_elements(a: int, b: int) -> int:
    """
    Computes the sum of two integers.

    Args:
        a (int): The first integer to be summed.
        b (int): The second integer to be summed.

    Returns:
        int: The sum of `a` and `b`.
    """
    return a + b


@tool
def multiply_two_elements(a: int, b: int) -> int:
    """
    Multiplies two integers.

    Args:
        a (int): The first integer to multiply.
        b (int): The second integer to multiply.

    Returns:
        int: The product of `a` and `b`.
    """
    return a * b

@tool
def compute_log(x: int) -> float | str:
    """
    Computes the logarithm of an integer `x` with an optional base.

    Args:
        x (int): The integer value for which the logarithm is computed. Must be greater than 0.

    Returns:
        float: The logarithm of `x` to the specified `base`.
    """
    if x <= 0:
        return "Logarithm is undefined for values less than or equal to 0."
    
    return math.log(x)


available_tools = {
    "sum_two_elements": sum_two_elements,
    "multiply_two_elements": multiply_two_elements,
    "compute_log": compute_log
}

Remember that the `@tool` operator allows us to convert a Python function into a `Tool` automatically. We cana check that very easily with some of the functions above.

In [ ]:
print("Tool name: ", sum_two_elements.name)
print("Tool signature: ", sum_two_elements.fn_signature)

### Adding the Tools signature to the System Prompt

Now, we just concatenate the tools signature and add them to the System Prompt.

In [ ]:
tools_signature = sum_two_elements.fn_signature + ",\n" + multiply_two_elements.fn_signature + ",\n" + compute_log.fn_signature

In [ ]:
print(tools_signature)

In [ ]:
REACT_SYSTEM_PROMPT = REACT_SYSTEM_PROMPT % tools_signature

In [ ]:
print(REACT_SYSTEM_PROMPT)

### ReAct Loop Step 1

In [ ]:
USER_QUESTION = "I want to calculate the sum of 1234 and 5678 and multiply the result by 5. Then, I want to take the logarithm of this result"
chat_history = [
    {
        "role": "system",
        "content": REACT_SYSTEM_PROMPT
    },
    {
        "role": "user",
        "content": f"<question>{USER_QUESTION}</question>"
    }
]


In [ ]:
output = GROQ_CLIENT.chat.completions.create(
    messages=chat_history,
    model=MODEL
).choices[0].message.content

print(output)

In [ ]:
chat_history.append(
    {
        "role": "assistant",
        "content": output
    }
)

### ReAct Loop Step 2

In [ ]:
tool_call = extract_tag_content(output, tag="tool_call")

In [ ]:
tool_call

In [ ]:
tool_call = json.loads(tool_call.content[0])

In [ ]:
tool_call

In [ ]:
tool_result = available_tools[tool_call["name"]].run(**tool_call["arguments"])

In [ ]:
assert tool_result == 1234 + 5678

In [ ]:
chat_history.append(
    {
        "role": "user",
        "content": f"<observation>{tool_result}</observation>"
    }
)

### ReAct Loop Step 3

In [ ]:
output = GROQ_CLIENT.chat.completions.create(
    messages=chat_history,
    model=MODEL
).choices[0].message.content

print(output)

In [ ]:
chat_history.append(
    {
        "role": "assistant",
        "content": output
    }
)

### ReAct Loop Step 4

In [ ]:
tool_call = extract_tag_content(output, tag="tool_call")
tool_call = json.loads(tool_call.content[0])
tool_result = available_tools[tool_call["name"]].run(**tool_call["arguments"])

In [ ]:
tool_result

In [ ]:
assert tool_result == (1234 + 5678) * 5

In [ ]:
chat_history.append(
    {
        "role": "user",
        "content": f"<observation>{tool_result}</observation>"
    }
)

### ReAct Loop Step 5

In [ ]:
output = GROQ_CLIENT.chat.completions.create(
    messages=chat_history,
    model=MODEL
).choices[0].message.content

print(output)

In [ ]:
chat_history.append(
    {
        "role": "assistant",
        "content": output
    }
)

### ReAct Loop Step 6

In [ ]:
tool_call = extract_tag_content(output, tag="tool_call")
tool_call = json.loads(tool_call.content[0])
tool_result = available_tools[tool_call["name"]].run(**tool_call["arguments"])

In [ ]:
tool_result

In [ ]:
assert tool_result == math.log((1234 + 5678) * 5)

In [ ]:
chat_history.append(
    {
        "role": "user",
        "content": f"<observation>{tool_result}</observation>"
    }
)

### ReAct Loop Step 7

In [ ]:
output = GROQ_CLIENT.chat.completions.create(
    messages=chat_history,
    model=MODEL
).choices[0].message.content

print(output)

## Doing the same but with `agentic_patterns` library

In [ ]:
from agentic_patterns.planning_pattern.react_agent import ReactAgent

In [ ]:
agent = ReactAgent(tools=[sum_two_elements, multiply_two_elements, compute_log])

In [ ]:
agent.run(user_msg="I want to calculate the sum of 1234 and 5678 and multiply the result by 5. Then, I want to take the logarithm of this result")

---

We did it!! A ReAct Agent working as expected, completely from Scratch! 🚀🚀🚀🚀